In [1]:
# 필요한 모듈 임포트
import pandas as pd
import tensorflow as tf
from keras.src.losses import loss
from tensorflow.keras import preprocessing
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dense, Dropout, Conv1D, GlobalMaxPool1D, concatenate



In [2]:
train_file = "total_train_data.csv"
data = pd.read_csv(train_file)
intents = data['고객의도'].tolist()
queries= data['고객질문(요청)'].tolist()



In [3]:
from utils.Preprocess import Preprocess
p = Preprocess(word2index_dic='C:\\Users\\wmk51\\PycharmProjects\\LB_project\\train_tools\\dict\\chatbot_dict.bin',
               userdic='C:\\Users\\wmk51\\PycharmProjects\\LB_project\\utils\\user_dic.tsv')

In [4]:
#단어 시퀀스 생성
sequences = []
for sentence in queries:
    pos = p.pos(sentence)
    keywords = p.get_keywords(pos, without_tag=True)
    seq = p.get_wordidx_sequence(keywords)
    sequences.append(seq)

In [17]:
#단어 인덱스 시퀀스 벡터 생성
#단어 시퀀스 벡터 생성
from config.GlobalParams import MAX_SEQ_LEN
padded_seqs = preprocessing.sequence.pad_sequences(sequences, maxlen=MAX_SEQ_LEN,padding='post')


In [25]:
#학습용, 검증용, 테스트용, 데이터셋
#학습셋:검증셋:테스트셋 = 7:2:1

ds = tf.data.Dataset.from_tensor_slices((padded_seqs,intents))
ds = ds.shuffle(len(queries))

train_size = int(len(padded_seqs)*0.7)
val_size = int(len(padded_seqs)*0.2)
test_size = len(padded_seqs)-train_size-val_size

train_ds = ds.take(train_size).batch(20)
val_ds = ds.skip(train_size).take(val_size).batch(20)
test_ds = ds.skip(train_size + val_size).take(test_size).batch(20)

#하이퍼파라미터 설정
dropout_prob = 0.5
EMB_SIZE = 128
EPOCH = 5
VOCAB_SIZE = len(p.word_index) + 1

#CNN 모델 정의
input_layer = Input(shape=(MAX_SEQ_LEN,))
embedding_layer = Embedding(VOCAB_SIZE,EMB_SIZE)(input_layer)
dropout_emb = Dropout(rate = dropout_prob)(embedding_layer)

conv1 = Conv1D(128, kernel_size=3, padding='valid', activation='relu')(dropout_emb)
pool1 = GlobalMaxPool1D()(conv1)

conv2 = Conv1D(128, kernel_size=3, padding='valid', activation='relu')(dropout_emb)
pool2 = GlobalMaxPool1D()(conv2)

conv3 = Conv1D(
    filters=128,
    kernel_size=5,
    padding= 'valid',
    activation = "relu"
)(dropout_emb)
pool3 = GlobalMaxPool1D()(conv3)

concat = concatenate([pool1,pool2,pool3])
hidden = Dense(128, activation='relu')(concat)
dropout_hidden = Dropout(rate=dropout_prob)(hidden)
logits = Dense(4, name='logits')(dropout_hidden)
predictions = Dense(4,activation='softmax')(logits)

#모델 생성
model = Model(inputs=input_layer, outputs=predictions)
## 타겟 데이터가 정수 이면 sparse_categorical_crossentropy 사용해야한다.
model.compile(optimizer="adam", loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#모델 학습

model.fit(train_ds, validation_data=val_ds, epochs=EPOCH, verbose=1)

#모델 평가
loss, accuracy = model.evaluate(test_ds,verbose=1)
print('Accuracy: %f' % (accuracy * 100))
print('loss: %f' % (loss))

model.save('intent_model.h5')

Epoch 1/5
18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 34ms/step - accuracy: 0.2873 - loss: 1.3535 - val_accuracy: 0.3469 - val_loss: 1.2131
Epoch 2/5
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.4631 - loss: 1.1520 - val_accuracy: 0.7245 - val_loss: 0.8397
Epoch 3/5
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6848 - loss: 0.8055 - val_accuracy: 0.8673 - val_loss: 0.5350
Epoch 4/5
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8062 - loss: 0.5804 - val_accuracy: 0.9184 - val_loss: 0.4004
Epoch 5/5
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8823 - loss: 0.4057 - val_accuracy: 0.9592 - val_loss: 0.2091
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step - accuracy: 0.9045 - loss: 0.2939


Accuracy: 92.156863
loss: 0.285709


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\wmk51\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\wmk51\AppData\Local\Temp\ipykernel_15388\2506353997.py", line 6, in <module>
    model.fit(train_ds, validation_date= val_ds, epochs=EPOCH, verbose=1)
  File "C:\Users\wmk51\anaconda3\envs\chatbot\Lib\site-packages\keras\src\utils\traceback_utils.py", line 122, in error_handler
    raise e.with_traceback(filtered_tb) from None
  File "C:\Users\wmk51\anaconda3\envs\chatbot\Lib\site-packages\keras\src\utils\traceback_utils.py", line 119, in error_handler
    filtered_tb = _process_traceback_frames(e.__traceback__)
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: TensorFlowTrainer.fit() got an unexpected keyword argument 'validation_date'

During handling of the above exception, another exception occurred:

Traceback (most recent call l